# US Census query limiations:
You can include up to 50 variables in a single API query and can make up to 500 queries per IP address per day. More than 500 queries per IP address per day requires that you register for a Census key. That key will be part of your data request URL string.

For python api:
https://pypi.org/project/census/

Your API key is "Hidden"

For variabler details   #https://api.census.gov/data/2016/acs/acs5/variables.html

 To understand how to pull the us censue data use this pdf:
 https://www.census.gov/content/dam/Census/programs-surveys/acs/guidance/training-presentations/20180614_API.pdf

 To get the varablie name use:
 https://api.census.gov/data/2016/acs/acs5/variables.html

 Tables I started with, may not be the complete list reviewed,  and may not be all tables that were used

 AIANHH = FIPS codes
 B00001_001E = UNWEIGHTED SAMPLE COUNT OF THE POPULATION
 B00002_001E = UNWEIGHTED SAMPLE HOUSING UNITS
 B19001_001E = HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)
 B01001_001E = SEX BY AGE
 B19019_001E = MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) BY HOUSEHOLD SIZE
 TRACT = Census Tract
 B06009_001E = PLACE OF BIRTH BY EDUCATIONAL ATTAINMENT IN THE UNITED STATES
 SDELM = School District (Elementary School)
 SDSEC = School District (Secondary School)
 STATE = State FIPS Code
 COUNTY = County FIPS Code
 CD115 = Congressional District
 C24050_001E = INDUSTRY BY OCCUPATION FOR THE CIVILIAN EMPLOYED POPULATION 16 YEARS AND OVER
 C24020_001E = SEX BY OCCUPATION FOR THE FULL-TIME, YEAR-ROUND CIVILIAN EMPLOYED POPULATION 16 YEARS AND OVER
 B20005_001E = SEX BY WORK EXPERIENCE IN THE PAST 12 MONTHS BY EARNINGS IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED         DOLLARS) FOR THE POPULATION 16 YEARS AND OVER
 B11016 = HOUSEHOLD TYPE BY HOUSEHOLD SIZE
 B11011 = HOUSEHOLD TYPE BY UNITS IN STRUCTURE
 B11001= HOUSEHOLD TYPE (INCLUDING LIVING ALONE)
 B11002 = HOUSEHOLD TYPE BY RELATIVES AND NONRELATIVES FOR POPULATION IN HOUSEHOLDS
 B09019 = HOUSEHOLD TYPE (INCLUDING LIVING ALONE) BY RELATIONSHIP

In [5]:
# @hidden_cell
ACCESS_TOKEN  = ''
token = ACCESS_TOKEN
url = "https://api.census.gov/data/2010/acs/acs5?get=NAME,B00001_001E,B00002_001E,B19001_001E&for=county:*&key="+ACCESS_TOKEN
write_file = (r'df.to_csv("C:/Users/Joe/us_cenusu_county_pop_2019.csv", index=False)')


In [2]:
# Goal - obtain all 3148 county data set for the year of 2000 to 2016, FIPS code,
# (demographics for that year later) population, income, housing count


# Extraction for year = 2010,2011,2012,2013,2014,2015,2016

from census import Census
from us import states
import numpy as np
import pandas as pd
import requests
import json

#There are two method to obtain the data
#data = c = Census("ACCESS_TOKEN", year = 2010)
#c.acs5.get('NAME,B00001_001E,B00002_001E,B19001_001E', {'for': 'county:*'})

# for loops do not split cells in jupyter, so doing it manually; years complete 2010
#url = "https://api.census.gov/data/"+year"+"/acs/acs5?get=NAME,B00001_001E,B00002_001E,B19001_001E&for=county:*&key="+your_us_census_access_code



JSONContent = requests.get(url).json()
content = json.dumps(JSONContent, indent = 4, sort_keys=True)  # loads or dump??
display(content)


'[\n    [\n        "NAME",\n        "B00001_001E",\n        "B00002_001E",\n        "B19001_001E",\n        "state",\n        "county"\n    ],\n    [\n        "Autauga County, Alabama",\n        "4564",\n        "1872",\n        "19718",\n        "01",\n        "001"\n    ],\n    [\n        "Baldwin County, Alabama",\n        "10965",\n        "5434",\n        "69476",\n        "01",\n        "003"\n    ],\n    [\n        "Barbour County, Alabama",\n        "2546",\n        "1105",\n        "9795",\n        "01",\n        "005"\n    ],\n    [\n        "Bibb County, Alabama",\n        "1254",\n        "574",\n        "7441",\n        "01",\n        "007"\n    ],\n    [\n        "Blount County, Alabama",\n        "4527",\n        "1913",\n        "20605",\n        "01",\n        "009"\n    ],\n    [\n        "Bullock County, Alabama",\n        "550",\n        "276",\n        "3732",\n        "01",\n        "011"\n    ],\n    [\n        "Butler County, Alabama",\n        "1673",\n        

In [3]:
# Transform 


df = pd.read_json(content, orient = 'columns')
df.columns = ['County, State', 's_count_pop', 's_housing_units', 'household_income', 's_FIPS', 'c_FIPS']

# split dataframe based on comma into two columns
# using an extra datframe, df_1, to split out state and county form NAME

df_1 =[]  
df_1 = df['County, State'].str.split(',', expand=True)
df[['County', 'State']] = df_1
df = df.iloc[1:,1:] # this is only way the zero row and zero column of data could be removed, other tried
df = df[['County','State','s_count_pop','s_housing_units', 'household_income','s_FIPS','c_FIPS']]
df["FIPS"] = "" # needed to add a column to the data frame
df.FIPS = df.s_FIPS + df.c_FIPS # tried a couple of ways , this worked and is the simpelest

# droping all data where state is greater than 57 becuse beyound this value are not US offical states
df.s_FIPS = pd.to_numeric(df.s_FIPS, errors='coerce')
df = df.loc[df['s_FIPS'] <= 80]

display(df.head())
display(df.tail())


,County,State,s_count_pop,s_housing_units,household_income,s_FIPS,c_FIPS,FIPS
1,Autauga County,Alabama,4564,1872,19718,1,001,01001
2,Baldwin County,Alabama,10965,5434,69476,1,003,01003
3,Barbour County,Alabama,2546,1105,9795,1,005,01005
4,Bibb County,Alabama,1254,574,7441,1,007,01007
5,Blount County,Alabama,4527,1913,20605,1,009,01009


,County,State,s_count_pop,s_housing_units,household_income,s_FIPS,c_FIPS,FIPS
3217,Vega Baja Municipio,Puerto Rico,4160,1833,18387,72,145,72145
3218,Vieques Municipio,Puerto Rico,619,336,3231,72,147,72147
3219,Villalba Municipio,Puerto Rico,1836,666,7512,72,149,72149
3220,Yabucoa Municipio,Puerto Rico,2812,1140,11908,72,151,72151
3221,Yauco Municipio,Puerto Rico,3571,1508,14080,72,153,72153


In [4]:
exec(write_file)
print(write_file)
          

df.to_csv("C:/Users/Joe/us_cenusu_county_pop_2019.csv", index=False)
